In [ ]:
# Import all necessary libraries
import pandas as pd
import torch
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import (
    BertTokenizer,
    BertForSequenceClassification
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
import os
from google.colab import files
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizerFast

In [ ]:

# Set environment variable for debugging CUDA errors
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Check device availability and set up properly
def setup_device():
    if torch.cuda.is_available():
        device = torch.device('cuda')
        print(f"Using GPU: {torch.cuda.get_device_name(0)}")
        print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
        # Clear cache to prevent memory issues
        torch.cuda.empty_cache()
    else:
        device = torch.device('cpu')
        print("Using CPU")
    return device

device = setup_device()

In [ ]:
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
CSV_FILE_PATH = "Training_clean.csv"  # Change this to your CSV file path
MODEL_NAME = 'bert-base-uncased'  # Use 'distilbert-base-uncased' for faster training
MAX_LENGTH = 128
BATCH_SIZE = 8  # Reduce to 8 or 4 if you get memory errors
EPOCHS = 3
LEARNING_RATE = 2e-5

# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

# Load data
# print("Loading dataset...")
df = pd.read_csv(CSV_FILE_PATH)
# print("Dataset loaded successfully!")
print(f"Dataset shape: {df.shape}")
# print("\ndfset columns:")
print(df.columns.tolist())
# print("\nFirst few rows:")
print(df.head())

# Check for missing values
print("\nMissing values:")
print(df.isnull().sum())

# Handle missing values
df = df.dropna()

In [ ]:
le = LabelEncoder()
df['label'] = le.fit_transform(df["type"])

train_text, val_text, train_label, val_label =train_test_split(
    df['tweet'].tolist(),
    df['label'].tolist(),
    test_size=0.1,
    random_state=42,
    stratify=df['label']
)

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

class TweetDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


In [ ]:
train_dataset = TweetDataset(train_text, train_label, tokenizer)
val_dataset = TweetDataset(val_text, val_label, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)


In [ ]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=len(le.classes_)
)
model.to(device)

In [ ]:
optimizer = optim.AdamW(model.parameters(), lr=2e-5)

In [ ]:
epochs = 3

for epoch in range(epochs):
    model.train()
    loop = tqdm(train_loader, leave=True)
    total_loss = 0
    for batch in loop:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        loop.set_description(f'Epoch {epoch+1}')
        loop.set_postfix(loss=loss.item())

    avg_train_loss = total_loss / len(train_loader)
    print(f'Average Training Loss: {avg_train_loss:.4f}')


In [ ]:
model.eval()
val_preds = []
val_true = []
with torch.no_grad():
    for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)

        val_preds.extend(preds.cpu().numpy())
        val_true.extend(labels.cpu().numpy())

print(f'Validation Classification Report Epoch {epoch+1}:')
print(classification_report(val_true, val_preds, target_names=le.classes_))


In [ ]:
# Save the model
SAVE_DIRECTORY = './save_gbv_bert_model'

model.save_pretrained(SAVE_DIRECTORY)
tokenizer.save_pretrained(SAVE_DIRECTORY)
print(f"Model saved to {SAVE_DIRECTORY}")

# Optional: Load model (for future use)
def load_saved_model(model_directory):
    """Load a previously saved model"""
    try:
        loaded_model = BertForSequenceClassification.from_pretrained(model_directory)
        loaded_tokenizer = BertTokenizer.from_pretrained(model_directory)
        loaded_model.to(device)
        print(f"Model loaded from {model_directory}")
        return loaded_model, loaded_tokenizer
    except Exception as e:
        print(f"Error loading model: {e}")
        return None

# Example of loading (uncomment to test)
# loaded_model, loaded_tokenizer = load_saved_model(SAVE_DIRECTORY)

In [ ]:
uploaded = files.upload()

# Assuming the uploaded file is named "Test.csv"
for filename in uploaded.keys():
    print(f'User uploaded file "{filename}" with length {len(uploaded[filename])} bytes')
    NEW_CSV_FILE_PATH = filename # Use the actual uploaded filename

new_data = pd.read_csv(NEW_CSV_FILE_PATH)

print("\nNew dataset loaded successfully!")
print(f"Dataset shape: {new_data.shape}")
print("\nFirst few rows of the new dataset:")
print(new_data.head())

In [ ]:
# Create a placeholder for labels for the new data
placeholder_labels = [0] * len(new_data)

# Create a dataset for the new data
new_dataset = TweetDataset(new_data['tweet'], placeholder_labels, tokenizer, MAX_LENGTH)

# Create a data loader for the new data
new_loader = DataLoader(new_dataset, batch_size=BATCH_SIZE, shuffle=False)

print("Data loader for new data created successfully!")
print(f"Number of batches in the new data loader: {len(new_loader)}")

# Test the new data loader
sample_batch_new = next(iter(new_loader))
print(f"\nSample batch shapes from new data loader:")
print(f"Input IDs: {sample_batch_new['input_ids'].shape}")
print(f"Attention Mask: {sample_batch_new['attention_mask'].shape}")
print(f"Labels: {sample_batch_new['labels'].shape}")

In [ ]:
# 1. Set the model to evaluation mode
model.eval()

# 2. Initialize an empty list called new_predictions to store the model's predictions.
new_predictions = []

print("Predicting on new data...")

# 3. Iterate through the new_loader using a for loop and tqdm for a progress bar.
for batch in tqdm(new_loader, desc="Predicting"):
    # 4. For each batch, move the input_ids and attention_mask to the appropriate device (device).
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)

    # 5. Disable gradient calculation using torch.no_grad().
    with torch.no_grad():
        # 6. Pass the input_ids and attention_mask to the model to get the outputs.
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

    # 7. Get the predicted class by finding the index of the maximum value in the outputs.logits along the last dimension.
    batch_predictions = torch.argmax(outputs.logits, dim=-1)

    # 8. Extend the new_predictions list with the predicted classes, converting them to NumPy arrays and moving them to the CPU.
    new_predictions.extend(batch_predictions.cpu().numpy())

print("\nPrediction complete!")

In [ ]:
# Create the label_map dictionary from the LabelEncoder classes
label_map = {i: label for i, label in enumerate(le.classes_)}

# 1. Invert the label_map dictionary to create a mapping from numerical IDs back to the original string labels.
id_to_label = {i: label for i, label in enumerate(le.classes_)}

# 2. Create an empty list called predicted_labels.
predicted_labels = []

# 3. Iterate through the new_predictions list (which contains the numerical predictions).
# 4. For each numerical prediction in new_predictions, use the id_to_label dictionary to find the corresponding string label.
# 5. Append the retrieved string label to the predicted_labels list.
for prediction in new_predictions:
    predicted_labels.append(id_to_label[prediction])

# 6. Print a message confirming that the numerical predictions have been mapped back to the original labels.
print("Numerical predictions have been mapped back to the original labels.")
print(f"Example predicted labels: {predicted_labels[:10]}")

In [ ]:
# 1. Create a new pandas DataFrame named results_df.
results_df = pd.DataFrame()

# 2. Add a column named 'Tweet_ID' to results_df and populate it with the 'Tweet_ID' column from the original new_data DataFrame.
results_df['Tweet_ID'] = new_data['Tweet_ID']

# 3. Add a column named 'Predicted_Type' to results_df and populate it with the predicted_labels list.
results_df['Predicted_Type'] = predicted_labels

# 4. Save results_df to a CSV file named 'predictions.csv'.
output_filename = 'predictions.csv'
results_df.to_csv(output_filename, index=False)

# 5. Print a confirmation message.
print(f"Prediction results saved to {output_filename}")
print("\nFirst few rows of the results DataFrame:")
print(results_df.head())

In [ ]:
#download the file to your local storage downloads folder
files.download('predictions.csv')